# Traffic in Bolzano

_NOTE_: This notebook, in order to properly work, should be uploaded in DataBricks, such that Spark can work and execute the following chunks of code.

Since credentials to connect to MySQL database should be kept secret, they are erased, but they can be found in the configuration file _config.yml_. 

## Connection to MySQL database

In [0]:
%scala
// Connection with MySQL DB
Class.forName("org.mariadb.jdbc.Driver")

val jdbcHostname = "traffic-db.ce2ieg6xrefy.us-east-2.rds.amazonaws.com"
val jdbcPort = 3306
val jdbcDatabase = "bluetoothstations"
var jdbcUsername = "marshall"
var jdbcPassword = "happyslashgiving"
// Create the JDBC URL without passing in the user and password parameters.
val jdbcUrl = s"jdbc:mysql://${jdbcHostname}:${jdbcPort}/${jdbcDatabase}"

// Create a Properties() object to hold the parameters.
import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", s"${jdbcUsername}")
connectionProperties.put("password", s"${jdbcPassword}")

import java.sql.DriverManager
val connection = DriverManager.getConnection(jdbcUrl, jdbcUsername, jdbcPassword)

jdbcHostname: String = traffic-db.ce2ieg6xrefy.us-east-2.rds.amazonaws.com
jdbcPort: Int = 3306
jdbcDatabase: String = bluetoothstations
jdbcUsername: String = marshall
jdbcPassword: String = happyslashgiving
jdbcUrl: String = jdbc:mysql://traffic-db.ce2ieg6xrefy.us-east-2.rds.amazonaws.com:3306/bluetoothstations
import java.util.Properties
connectionProperties: java.util.Properties = {user=marshall, password=happyslashgiving}
import java.sql.DriverManager
connection: java.sql.Connection = org.mariadb.jdbc.MariaDbConnection@131777ae

In [0]:
%scala
// Opening the connection to measurement table inside the DB
val measurement = spark.read.jdbc(jdbcUrl, "measurement", connectionProperties)

// Opening the connection to station table inside the DB
val station = spark.read.jdbc(jdbcUrl, "station", connectionProperties)

measurement: org.apache.spark.sql.DataFrame = [timestamp: timestamp, count: int ... 1 more field]
station: org.apache.spark.sql.DataFrame = [code: int, name: string ... 2 more fields]

In [0]:
%scala 
// Create temporary view of tables to pass data from scala to python
measurement.createOrReplaceTempView("measurement")
station.createOrReplaceTempView("station")

## Train and test splitting

In [0]:
!pip install keras
!pip install tensorflow

Collecting keras
 Downloading Keras-2.4.3-py2.py3-none-any.whl (36 kB)
Collecting pyyaml
 Downloading PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
▌ | 10 kB 21.3 MB/s eta 0:00:01 |█ | 20 kB 16.3 MB/s eta 0:00:01 |█▌ | 30 kB 7.9 MB/s eta 0:00:01 |██ | 40 kB 10.0 MB/s eta 0:00:01 |██▌ | 51 kB 5.0 MB/s eta 0:00:01 |███ | 61 kB 5.9 MB/s eta 0:00:01 |███▌ | 71 kB 6.2 MB/s eta 0:00:01 |████ | 81 kB 6.4 MB/s eta 0:00:01 |████▌ | 92 kB 7.1 MB/s eta 0:00:01 |█████ | 102 kB 7.0 MB/s eta 0:00:01 |█████▍ | 112 kB 7.0 MB/s eta 0:00:01 |██████ | 122 kB 7.0 MB/s eta 0:00:01 |██████▍ | 133 kB 7.0 MB/s eta 0:00:01 |███████ | 143 kB 7.0 MB/s eta 0:00:01 |███████▍ | 153 kB 7.0 MB/s eta 0:00:01 |████████ | 163 kB 7.0 MB/s eta 0:00:01 |████████▍ | 174 kB 7.0 MB/s eta 0:00:01 |█████████ | 184 kB 7.0 MB/s eta 0:00:01 |█████████▍ | 194 kB 7.0 MB/s eta 0:00:01 |██████████ | 204 kB 7.0 MB/s eta 0:00:01 |██████████▍ | 215 kB 7.0 MB/s eta 0:00:01 |██████████▉ | 225 kB 7.0 MB/s eta 0:00:01 |███████████▍ | 235 kB 7.0 MB/s eta 0:00:01 |███████████▉ | 245 kB 7.0 MB/s eta 0:00:01 |████████████▍ | 256 kB 7.0 MB/s eta 0:00:01 |████████████▉ | 266 kB 7.0 MB/s eta 0:00:01 |█████████████▍ | 276 kB 7.0 MB/s eta 0:00:01 |█████████████▉ | 286 kB 7.0 MB/s eta 0:00:01 |██████████████▍ | 296 kB 7.0 MB/s eta 0:00:01 |██████████████▉ | 307 kB 7.0 MB/s eta 0:00:01 |███████████████▍ | 317 kB 7.0 MB/s eta 0:00:01 |███████████████▉ | 327 kB 7.0 MB/s eta 0:00:01 |████████████████▎ | 337 kB 7.0 MB/s eta 0:00:01 |████████████████▉ | 348 kB 7.0 MB/s eta 0:00:01 |█████████████████▎ | 358 kB 7.0 MB/s eta 0:00:01 |█████████████████▉ | 368 kB 7.0 MB/s eta 0:00:01 |██████████████████▎ | 378 kB 7.0 MB/s eta 0:00:01 |██████████████████▉ | 389 kB 7.0 MB/s eta 0:00:01 |███████████████████▎ | 399 kB 7.0 MB/s eta 0:00:01 |███████████████████▉ | 409 kB 7.0 MB/s eta 0:00:01 |████████████████████▎ | 419 kB 7.0 MB/s eta 0:00:01 |████████████████████▊ | 430 kB 7.0 MB/s eta 0:00:01 |█████████████████████▎ | 440 kB 7.0 MB/s eta 0:00:01 |█████████████████████▊ | 450 kB 7.0 MB/s eta 0:00:01 |██████████████████████▎ | 460 kB 7.0 MB/s eta 0:00:01 |██████████████████████▊ | 471 kB 7.0 MB/s eta 0:00:01 |███████████████████████▎ | 481 kB 7.0 MB/s eta 0:00:01 |███████████████████████▊ | 491 kB 7.0 MB/s eta 0:00:01 |████████████████████████▎ | 501 kB 7.0 MB/s eta 0:00:01 |████████████████████████▊ | 512 kB 7.0 MB/s eta 0:00:01 |█████████████████████████▎ | 522 kB 7.0 MB/s eta 0:00:01 |█████████████████████████▊ | 532 kB 7.0 MB/s eta 0:00:01 |██████████████████████████▏ | 542 kB 7.0 MB/s eta 0:00:01 |██████████████████████████▊ | 552 kB 7.0 MB/s eta 0:00:01 |███████████████████████████▏ | 563 kB 7.0 MB/s eta 0:00:01 |███████████████████████████▊ | 573 kB 7.0 MB/s eta 0:00:01 |████████████████████████████▏ | 583 kB 7.0 MB/s eta 0:00:01 |████████████████████████████▊ | 593 kB 7.0 MB/s eta 0:00:01 |█████████████████████████████▏ | 604 kB 7.0 MB/s eta 0:00:01 |█████████████████████████████▊ | 614 kB 7.0 MB/s eta 0:00:01 |██████████████████████████████▏ | 624 kB 7.0 MB/s eta 0:00:01 |██████████████████████████████▊ | 634 kB 7.0 MB/s eta 0:00:01 |███████████████████████████████▏| 645 kB 7.0 MB/s eta 0:00:01 |███████████████████████████████▋| 655 kB 7.0 MB/s eta 0:00:01 |████████████████████████████████| 662 kB 7.0 MB/s 
Requirement already satisfied: scipy>=0.14 in /databricks/python3/lib/python3.8/site-packages (from keras) (1.5.2)
Collecting h5py
 Downloading h5py-3.3.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.5 MB)
 | 10 kB 23.9 MB/s eta 0:00:01 |▏ | 20 kB 29.5 MB/s eta 0:00:01 |▏ | 30 kB 35.3 MB/s eta 0:00:01 |▎ | 40 kB 26.7 MB/s eta 0:00:01 |▍ | 51 kB 15.0 MB/s eta 0:00:01 |▍ | 61 kB 17.1 MB/s eta 0:00:01 |▌ | 71 kB 16.5 MB/s eta 0:00:01 |▋ | 81 kB 18.0 MB/s eta 0:00:01 |▋ | 92 kB 17.5 MB/s eta 0:00:01 |▊ | 102 kB 12.3 MB/s eta 0:00:01 |▉ | 112 kB 12.3 MB/s eta 0:00:01 |▉ | 122 kB 12.3 MB/s eta 0:00

In [0]:
%python
# Libraries

## Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras

## Data Transformation
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from numpy import array
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## Query
from pyspark.sql.functions import hour, mean
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [0]:
%python
# Creating tables in Python
df = spark.table("measurement")
stations = spark.table('station')

# Dropping useless information for the model
station_codes = stations.drop("longitude","latitude")

# Joining measurements and station to map stations as numerical code
df = df.join(station_codes,df.station == stations.name)\
.drop("name")

In [0]:
# Splitting data into training and testing based on the latest timestamp inside the DB
latest_timestamp = df.agg({"timestamp":"max"}).collect()[0]

# Data between latest timestamp and 2 days before that
test_subset = df.filter(df["timestamp"] >= latest_timestamp[0]-timedelta(hours=168))
# Oldest data till 2 days before the latest timestamp
train_subset = df.filter(df["timestamp"] < latest_timestamp[0]-timedelta(hours=168))

## Model for predicting traffic

In [0]:
# Create a subset of dataframe for each different station
def prepare_dataset_for_sequential(dataframe, last_date, stations):
    data = dataframe.toPandas()
    # Create a list of dataframe for each station
    codes = dict()
    for i in range(len(stations)):
        codes[stations[i]] = i+1
    data['station'] = [codes[x] for x in data['station']]
    data_per_station = [data[data['station']== x] for x in range(1,len(stations)+1)]
    data = data[['count','timestamp','station']]
    return data_per_station, codes

In [0]:
# Consider 5 temporal stages of data
def create_model_dataset(dataframes):
  scaler = StandardScaler()
  final_df = pd.DataFrame()
  for df in dataframes:
    if not df.empty:
      count = scaler.fit_transform(df['count'].values.astype('float32').reshape(1,-1).T)
      scaled = np.concatenate((count,df['station'].values.astype('float32').reshape(1,-1).T), axis=1)
      # frame as supervised learning
      reframed = series_to_supervised(scaled, 5, 1)
      reframed.drop(['var2(t)'], axis=1, inplace=True)
      final_df = final_df.append(reframed)
  return final_df.reset_index(drop=True), scaler

In [0]:
# Convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [0]:
def data_split(data):
    train_X = data[:,:-1]
    train_y = data[:,-1]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    return train_X, train_y

In [0]:
stations_names = [x[0] for x in stations.select("name").collect()]

In [0]:
%python
# Train preparation
data_per_station_train, codes = prepare_dataset_for_sequential(train_subset, latest_timestamp, stations_names)
train, scaler_train = create_model_dataset(data_per_station_train)
train = train.values
train_X, train_y = data_split(train)

# Test preparation
data_per_station_test, _ = prepare_dataset_for_sequential(test_subset, latest_timestamp, stations_names)
test, scaler_test = create_model_dataset(data_per_station_test)
test = test.values
test_X, test_y = data_split(test)

In [0]:
# Preview of train sequential dataset
train

Out[11]: array([[-4.8973238e-01, 1.0000000e+00, -4.8973238e-01, ...,
 -4.8973238e-01, 1.0000000e+00, -4.8973238e-01],
 [-4.8973238e-01, 1.0000000e+00, -4.8973238e-01, ...,
 -4.8973238e-01, 1.0000000e+00, -4.8973238e-01],
 [-4.8973238e-01, 1.0000000e+00, -4.8973238e-01, ...,
 -4.8973238e-01, 1.0000000e+00, -4.8973238e-01],
 ...,
 [ 3.2936257e-01, 7.0000000e+01, 1.8846390e-01, ...,
 -4.2485976e-01, 7.0000000e+01, -4.2485976e-01],
 [ 1.8846390e-01, 7.0000000e+01, -1.8740037e-02, ...,
 -4.2485976e-01, 7.0000000e+01, 3.1278625e-01],
 [-1.8740037e-02, 7.0000000e+01, 7.0924993e+00, ...,
 3.1278625e-01, 7.0000000e+01, -4.2485976e-01]], dtype=float32)

In [0]:
# Neural network with LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=2, batch_size=72, 
                    validation_data=(test_X, test_y), 
                    verbose=2, shuffle=False)

Epoch 1/2
188774/188774 - 410s - loss: 0.1601 - val_loss: 0.8077
Epoch 2/2
188774/188774 - 386s - loss: 0.1374 - val_loss: 0.5005

In [0]:
y_hat = model.predict(test_X)

In [0]:
# To better visualize predictions
def obtain_prediction_dataframe(model, X, data, scaler, station_codes):
    yhat = model.predict(X)
    predictions = scaler.inverse_transform(yhat)

    indexes = dict()
    for label in station_codes.values():
        indexes[label] = 0
    i = 1
    for j in range(len(data[:,1])-1):
        if data[j,1]>i:
            indexes[data[j-1,1]] = predictions.ravel()[j-1]
            i = i+1
        
    indexes[list(indexes.keys())[-1]] = predictions.ravel()[-1]
    preds = indexes.values()
    # 9. Creating the output csv
    output = pd.DataFrame()
    output['count'] = [max(int(np.round(x,0)),0) for x in preds]
    output['station'] = station_codes.keys()
    output['timestamp'] = [(latest_timestamp[0]+timedelta(minutes=10))] * len(preds)
    return output

predictions = obtain_prediction_dataframe(model, test_X, test, scaler_test, codes)

In [0]:
predictions

Out[15]:

,count,station,timestamp
0,124,A22_ML103,2021-07-30 09:40:00
1,31,A22_ML107,2021-07-30 09:40:00
2,61,Agip_Einstein,2021-07-30 09:40:00
3,27,Arginale_Palermo,2021-07-30 09:40:00
4,168,Arginale_Resia,2021-07-30 09:40:00
...,...,...,...
65,0,Vienna,2021-07-30 09:40:00
66,0,Vittorio veneto,2021-07-30 09:40:00
67,49,wolf,2021-07-30 09:40:00
68,50,wolf-franziskus,2021-07-30 09:40:00


### Save the ML model

In [0]:
# Save model structure
model_json = model.to_json()
with open("/tmp/model.json", "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
model.save_weights("/tmp/model.h5")

# Move files to filestore (accessible with files/model/...)
dbutils.fs.cp('file:/tmp/model.h5', "dbfs:/FileStore/model/model.h5")
dbutils.fs.cp('file:/tmp/model.json', "dbfs:/FileStore/model/model.json")

Out[16]: True

## Evaluate the model

In [0]:
last_timestamp = datetime.strptime(str(predictions['timestamp'].unique()[0]), "%Y-%m-%dT%H:%M:%S.%f000")
latest_results = df.filter(df.timestamp >= last_timestamp).filter(df.timestamp <= last_timestamp+timedelta(minutes=10)).sort(df.timestamp).toPandas()
latest_results = latest_results.rename(columns = {'count':'actual'})
minimum = min(latest_results['timestamp'])
latest_results = latest_results[latest_results['timestamp'] == minimum].drop('timestamp',axis = 1)
results = predictions.set_index('station').join(latest_results.set_index('station'))

# Set NAN values of missing stations to 0
for i in range(len(results['actual'])):
  if np.isnan(results['actual'][i]):
    results['actual'][i] = 0
    
results = results.drop('code',axis = 1)

<command-3060370921840364>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 results['actual'][i] = 0

In [0]:
# Mean Squared Error
np.sum((y_hat.reshape(1,-1)-test_y)**2)


Out[19]: 76396.87

In [0]:
# Root Mean Squared Error
np.sqrt(np.sum((y_hat.reshape(1,-1)-test_y)**2))


Out[21]: 276.39984

In [0]:
# Mean Absolute Error
np.mean(abs(y_hat.reshape(1,-1)-test_y))

Out[20]: 0.5005

In [0]:
# Visual results
pd.set_option('display.max_rows', None)
results

Out[159]:

,count,timestamp,actual
station,,,
A22_ML103,0,2021-07-29 16:40:00,303.0
A22_ML107,0,2021-07-29 16:40:00,380.0
Agip_Einstein,103,2021-07-29 16:40:00,200.0
Arginale_Palermo,263,2021-07-29 16:40:00,316.0
Arginale_Resia,209,2021-07-29 16:40:00,314.0
Buozzi_Fiera,91,2021-07-29 16:40:00,136.0
caugusta,126,2021-07-29 16:40:00,110.0
cavour,0,2021-07-29 16:40:00,143.0
cfirmiano,0,2021-07-29 16:40:00,0.0
